In [1]:
#FROM CHATGPT :)

import win32com.client
from collections import defaultdict
from bs4 import BeautifulSoup
from transformers import pipeline
import logging

logging.basicConfig(level=logging.ERROR, filename="error_log.txt")

summarizer = pipeline("summarization")

def connect_to_outlook():
    outlook = win32com.client.Dispatch("Outlook.Application")
    namespace = outlook.GetNamespace("MAPI")
    return namespace.GetDefaultFolder(6)  # Inbox folder

def get_body_preview(body):
    try:
        text = BeautifulSoup(body, "html.parser").get_text()
        return text[:50] + "..." if len(text) > 50 else text
    except Exception as e:
        logging.error(f"Error parsing body: {e}")
        return "Error reading body"

def summarize_thread(emails):
    full_body = " ".join(email['body'] for email in emails)
    try:
        return summarizer(full_body, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
    except Exception as e:
        logging.error(f"Error summarizing thread: {e}")
        return "Error generating summary"

def fetch_emails(inbox):
    threads = defaultdict(list)
    for message in inbox.Items:
        try:
            if message.Class == 43:  # MailItem
                threads[message.Subject].append({
                    'sender': message.SenderName,
                    'body': message.Body,
                    'received_time': message.ReceivedTime
                })
        except Exception as e:
            logging.error(f"Error processing message: {e}")
    return threads

def display_threads(email_threads):
    for subject, emails in email_threads.items():
        print(f"Subject: {subject} ({len(emails)} emails)")
        print("=" * 50)
        emails.sort(key=lambda email: email['received_time'])
        for email in emails:
            print(f"- From: {email['sender']} | Received: {email['received_time']}")
            print(f"  Preview: {get_body_preview(email['body'])}")
        print(f"Thread Summary: {summarize_thread(emails)}")
        print("\n")

def summarize_email_thread():
    inbox = connect_to_outlook()
    email_threads = fetch_emails(inbox)
    display_threads(email_threads)

if __name__ == "__main__":
    summarize_email_thread()


ModuleNotFoundError: No module named 'transformers'

In [21]:
#GITHUB CODE TEST

import imaplib
import email
import regex as re
import codecs
import string

# create an IMAP4_SSL class instance
imap = imaplib.IMAP4_SSL("imap.gmail.com")

# login with your Gmail account credentials
username = "qihengaw@gmail.com"
password = "ynan gdtm tqfr hpqu" #shoul be an app password, to create one follow this link:https://support.google.com/accounts/answer/185833
imap.login(username, password)

for i in imap.list()[1]:
    l = i.decode().split(' "/" ')
    print(l[0] + " = " + l[1])

imap.select('"INBOX"')
status, messages = imap.search(None, 'UNSEEN')

# iterate over the messages and retrieve their contents
for num in messages[0].split()[::-1]:
    _, msg = imap.fetch(num, "(RFC822)")
    message = email.message_from_bytes(msg[0][1])
    if message.is_multipart():
      for part in message.walk():
        if part.get_content_type() == "text/plain":
            content = part.get_payload(decode=True)
            body=content.decode("utf-8")
            break
        else:
        # if no text/plain part was found, use the first part as the body
          content = message.get_payload(0).get_payload(decode=True)
          body=content.decode("utf-8")
    else:
    # if the message is not multipart, just use the message body
      content = message.get_payload(decode=True)
      body=content.decode("utf-8")

    # print the message details
    subject_header = message['Subject']
    decoded_subject = email.header.decode_header(subject_header)
    subject = decoded_subject[0][0]
    print("Subject:", subject)
    print("From:", message["From"])
    print("Date:", message["Date"])
    print("Body", body)


from transformers import pipeline, AutoTokenizer
summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
text = '''The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.
'''
summarizer(text)

summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY", tokenizer=AutoTokenizer.from_pretrained("knkarthick/MEETING_SUMMARY"))

imap.select('"INBOX"')
status, messages = imap.search(None, 'UNSEEN')
summaryofsentemails=[]

# iterate over the messages and retrieve their contents
for num in messages[0].split()[::-1]:
    _, msg = imap.fetch(num, "(RFC822)")
    message = email.message_from_bytes(msg[0][1])

    # Handle multipart emails
    if message.is_multipart():
        body = None
        for part in message.walk():
            if part.get_content_type() == "text/plain":  # Plain text part
                content = part.get_payload(decode=True)
                if content:
                    try:
                        body = content.decode("utf-8")
                    except UnicodeDecodeError:
                        body = content.decode("latin1")
                    break
    else:
        content = message.get_payload(decode=True)
        if content:
            try:
                body = content.decode("utf-8")
            except UnicodeDecodeError:
                body = content.decode("latin1")
        else:
            body = "No content available"

    # Ensure body is valid
    if not body:
        body = "No content available"

    # Process the email details
    subject_header = message['Subject']
    decoded_subject = email.header.decode_header(subject_header)
    subject = decoded_subject[0][0]
    if isinstance(subject, bytes):
        subject = subject.decode(decoded_subject[0][1] or "utf-8")

    print("Subject:", subject)
    print("From:", message["From"])
    print("Date:", message["Date"])

    # Generate the summary
    try:
        content_summarised = summarizer(body, max_length=100, min_length=10, truncation=True)[0]['summary_text']
        print("Summary:", content_summarised)
    except Exception as e:
        print(f"Error summarizing email: {e}")
        content_summarised = "Unable to generate summary"

    # Append to summary list
    summary = {
        "Subject": subject,
        "From": str(message["From"]),
        "Date": str(message["Date"]),
        "Email": content_summarised,
    }
    summaryofsentemails.append(summary)



(\HasNoChildren) = "INBOX"
(\HasChildren \Noselect) = "[Gmail]"
(\All \HasNoChildren) = "[Gmail]/All Mail"
(\Drafts \HasNoChildren) = "[Gmail]/Drafts"
(\HasNoChildren \Important) = "[Gmail]/Important"
(\HasNoChildren \Sent) = "[Gmail]/Sent Mail"
(\HasNoChildren \Junk) = "[Gmail]/Spam"
(\Flagged \HasNoChildren) = "[Gmail]/Starred"
(\HasNoChildren \Trash) = "[Gmail]/Trash"
Subject: Your FairPrice Group app receipt
From: FairPrice <no-reply@fairprice.com.sg>
Date: Thu, 31 Oct 2024 11:14:08 +0000
Body <!DOCTYPE html>
<html lang="en">
   <head>
      
      <meta charset="UTF-8">
      <meta content="width=device-width,initial-scale=1" name="viewport">
      <meta name="x-apple-disable-message-reformatting">
      <meta http-equiv="X-UA-Compatible" content="IE=edge">
      <meta content="telephone=no" name="format-detection">
      <title>Your FairPrice Group receipt</title>
      <style>
         * {
         -webkit-text-size-adjust: none;
         }
         body {
         font-family: 

AttributeError: 'NoneType' object has no attribute 'decode'

In [19]:
summaryofsentemails

NameError: name 'summaryofsentemails' is not defined